In [34]:
import numpy as np
import pandas as pd

class Instancia():
    def __init__(self,csvfile):
        self._equipos=0
        self._partidos=0
        self._cmm = None
        self.mass=None
        self.diferencia=None
        self.ganados = None
        self.totales = None
        with open(csvfile) as f:
            lines = f.read().split('\n')[:-1]
            for i, line in enumerate(lines):
                if i == 0: # header
                    columns = line.split()
                    self._equipos = int(columns[0])
                    self._partidos = int(columns[1])
                    self._cmm = np.empty([self._equipos,self._equipos])
                    self.mass = np.empty([self._equipos,self._equipos])
                    self.ganados= np.empty([self._equipos,self._equipos])
                    self.totales = np.zeros(self._equipos)
                    self.diferencias = np.zeros(self._equipos)
                    for i in range(0,self._equipos):
                        for j in range(0,self._equipos):
                            self.ganados[i,j]=0.0
                else:
                    data = line.split();
                    print(data)
#                     set_trace()
                    self.totales[int(data[1])-1]+=1
                    self.totales[int(data[3])-1]+=1
        
                    self.diferencias[int(data[1])-1]+=int(data[2])
                    self.diferencias[int(data[1])-1]-=int(data[4])
                    
                    
                    self.diferencias[int(data[3])-1]+=int(data[4])
                    self.diferencias[int(data[3])-1]-=int(data[2])
                    
                    if(int(data[2])>int(data[4])):
                        self.ganados[int(data[1])-1,int(data[3])-1] +=1
                    else:
                        self.ganados[int(data[3])-1,int(data[1])-1] +=1
                    
    def equipos(self):
        return self._equipos
    def partidos(self):
        return self._partidos
    def cmm(self):
        return self._cmm

In [35]:
ins = Instancia("tests/test-prob-2.in")
            

['1', '1', '16', '4', '13']
['1', '2', '38', '5', '17']
['1', '2', '0', '6', '1']
['1', '3', '34', '1', '21']
['1', '3', '23', '4', '10']
['1', '4', '31', '1', '6']
['1', '5', '0', '6', '1']
['1', '5', '38', '4', '23']
['1', '6', '1', '2', '0']
['1', '6', '1', '5', '0']
['1', '4', '1', '6', '0']
['1', '5', '1', '6', '0']


In [36]:
ins.partidos()

12

In [37]:
ins.diferencias

array([-35.,  19.,  26.,  -5.,  -7.,   2.])

In [38]:
ins.totales

array([3., 3., 2., 5., 5., 6.])

In [39]:
for i in range(0, ins.equipos()):
    for j in range(0, ins.equipos()):
        if i==j:
            ins._cmm[i][j]=  ins.totales[i]+ 2
            ins.mass[i][j]= ins.totales[i]
        else:
            ins._cmm[i][j]=  (ins.ganados[i][j]+ ins.ganados[j][i])*-1
            ins.mass[i][j]= (ins.ganados[i][j]+ ins.ganados[j][i])*-1

In [40]:
ins._cmm

array([[ 5., -0., -1., -2., -0., -0.],
       [-0.,  5., -0., -0., -1., -2.],
       [-1., -0.,  4., -1., -0., -0.],
       [-2., -0., -1.,  7., -1., -1.],
       [-0., -1., -0., -1.,  7., -3.],
       [-0., -2., -0., -1., -3.,  8.]])

In [41]:
ins.mass

array([[ 3., -0., -1., -2., -0., -0.],
       [-0.,  3., -0., -0., -1., -2.],
       [-1., -0.,  2., -1., -0., -0.],
       [-2., -0., -1.,  5., -1., -1.],
       [-0., -1., -0., -1.,  5., -3.],
       [-0., -2., -0., -1., -3.,  6.]])

In [42]:
b = np.zeros(ins.equipos())

In [43]:
for i in range(0,ins.equipos()):
    ganados =0
    for j in range(0,ins.equipos()):
        ganados += ins.ganados[i][j]
    b[i]= 1+(2*ganados-ins.totales[i])/2

In [44]:
b

array([0.5, 0.5, 2. , 0.5, 0.5, 2. ])

In [45]:
import math


def forward_elimination(A, b, n):
    """
    Calculates the forward part of Gaussian elimination.
    """
    for row in range(0, n-1):
        for i in range(row+1, n):
            factor = A[i,row] / A[row,row]
            for j in range(row, n):
                A[i,j] = A[i,j] - factor * A[row,j]

            b[i] = b[i] - factor * b[row]

        print('A = \n%s and b = %s' % (A,b))
    return A, b

def back_substitution(a, b, n):
    """"
    Does back substitution, returns the Gauss result.
    """
    x = np.zeros((n,1))
    x[n-1] = b[n-1] / a[n-1, n-1]
    for row in range(n-2, -1, -1):
        sums = b[row]
        for j in range(row+1, n):
            sums = sums - a[row,j] * x[j]
        x[row] = sums / a[row,row]
    return x

def gauss(A, b):
    """
    This function performs Gauss elimination without pivoting.
    """
    n = A.shape[0]

    # Check for zero diagonal elements
    if any(np.diag(A)==0):
        raise ZeroDivisionError(('Division by zero will occur; '
                                  'pivoting currently not supported'))

    A, b = forward_elimination(A, b, n)
    return back_substitution(A, b, n)

In [46]:
eg = gauss(ins.mass,ins.diferencias) 

A = 
[[ 3.         -0.         -1.         -2.         -0.         -0.        ]
 [ 0.          3.         -0.         -0.         -1.         -2.        ]
 [ 0.         -0.          1.66666667 -1.66666667 -0.         -0.        ]
 [ 0.         -0.         -1.66666667  3.66666667 -1.         -1.        ]
 [ 0.         -1.         -0.         -1.          5.         -3.        ]
 [ 0.         -2.         -0.         -1.         -3.          6.        ]] and b = [-35.          19.          14.33333333 -28.33333333  -7.
   2.        ]
A = 
[[ 3.         -0.         -1.         -2.         -0.         -0.        ]
 [ 0.          3.         -0.         -0.         -1.         -2.        ]
 [ 0.          0.          1.66666667 -1.66666667 -0.         -0.        ]
 [ 0.          0.         -1.66666667  3.66666667 -1.         -1.        ]
 [ 0.          0.         -0.         -1.          4.66666667 -3.66666667]
 [ 0.          0.         -0.         -1.         -3.66666667  4.66666667]] and b =

In [48]:
eg

array([[-12.22],
       [ 10.22],
       [  5.18],
       [ -3.42],
       [  2.66],
       [  4.5 ]])

In [49]:
eg2 = gauss(ins._cmm,b) 

A = 
[[ 5.  -0.  -1.  -2.  -0.  -0. ]
 [ 0.   5.  -0.  -0.  -1.  -2. ]
 [ 0.  -0.   3.8 -1.4 -0.  -0. ]
 [ 0.  -0.  -1.4  6.2 -1.  -1. ]
 [ 0.  -1.  -0.  -1.   7.  -3. ]
 [ 0.  -2.  -0.  -1.  -3.   8. ]] and b = [0.5 0.5 2.1 0.7 0.5 2. ]
A = 
[[ 5.  -0.  -1.  -2.  -0.  -0. ]
 [ 0.   5.  -0.  -0.  -1.  -2. ]
 [ 0.   0.   3.8 -1.4 -0.  -0. ]
 [ 0.   0.  -1.4  6.2 -1.  -1. ]
 [ 0.   0.  -0.  -1.   6.8 -3.4]
 [ 0.   0.  -0.  -1.  -3.4  7.2]] and b = [0.5 0.5 2.1 0.7 0.6 2.2]
A = 
[[ 5.         -0.         -1.         -2.         -0.         -0.        ]
 [ 0.          5.         -0.         -0.         -1.         -2.        ]
 [ 0.          0.          3.8        -1.4        -0.         -0.        ]
 [ 0.          0.          0.          5.68421053 -1.         -1.        ]
 [ 0.          0.          0.         -1.          6.8        -3.4       ]
 [ 0.          0.          0.         -1.         -3.4         7.2       ]] and b = [0.5        0.5        2.1        1.47368421 0.6        2.2 

In [50]:
eg2

array([[0.41778266],
       [0.41745335],
       [0.71383095],
       [0.43754116],
       [0.43951701],
       [0.57387486]])

In [52]:
m_row = 0# current row in Massey matrix

# also build point differential vector p
p = np.zeros((num_teams,1))

# Colley approach requires b, win differential basically
b = np.zeros((num_teams,1))

for k in team_name_list :
    # find all games team played, win or loss
    team_games = reg2011_df[(reg2011_df['Wteam_name'] == k) | (reg2011_df['Lteam_name'] == k)]
    n_team = np.shape(team_games)[0]
    # Number of games team played, for diagonal

    times_played = np.zeros((1,np.shape(M)[1])) # basically an array for given row in M
    
    # counter for array indexing
    ctr = 0
    for j in team_name_list :
        if j == k :
            # this is the team itself
            times_played[0,ctr] = n_team
        else :
            # Find all matches between team of interest (k) and opponent (j)
            matches = team_games[(team_games['Wteam_name'] == j) | (team_games['Lteam_name'] == j)]
            times_played[0,ctr] = np.negative(matches.shape[0])
        ctr += 1
    
    # now add current row to M
    M[m_row] = times_played
    
    # add cumulative points to p
    # p_wins is point differential in games won by team k
    p_wins = np.sum(\
        team_games[team_games['Wteam_name'] == k]['Wscore'] - \
            team_games[team_games['Wteam_name'] == k]['Lscore']
    )
    # p_losses is point differential in games lost by team k (will be negative)
    p_losses = np.sum(\
        team_games[team_games['Lteam_name'] == k]['Lscore'] - \
            team_games[team_games['Lteam_name'] == k]['Wscore']
    )
    p[m_row] = p_wins + p_losses
    
    # Now build Colley right-hand side vector b
    b[m_row] = 1 + (0.5) * (\
        team_games[team_games['Wteam_name'] == k].shape[0] - \
            team_games[team_games['Lteam_name'] == k].shape[0]
    )
    
    # iterate to next row in M
    m_row += 1

NameError: name 'team_name_list' is not defined